https://colab.research.google.com/github/daangn/notebooks/blob/master/daangn/profile_image_detector_using_tf_hub.ipynb#scrollTo=CKXcZN1KkysM

참고로 tensorflow_hub 를 통해서 유사 이미지를 검색 하는 학습을 하고, 그를 기반으로 유사 이미지를 찾아내는 코딩을 테스트 해본다.

In [1]:
!pip -q install tensorflow-hub

import tensorflow as tf
import tensorflow_hub as hub

CHANNELS = 3 # number of image channels (RGB)

def build_graph(hub_module_url, target_image_path):
  # Step 1) Prepare pre-trained model for extracting image features.
  module = hub.Module(hub_module_url)
  height, width = hub.get_expected_image_size(module)

  # Copied a method of https://github.com/GoogleCloudPlatform/cloudml-samples/blob/bf0680726/flowers/trainer/model.py#L181
  # and fixed for all type images (not only jpeg)
  def decode_and_resize(image_str_tensor):
    """Decodes jpeg string, resizes it and returns a uint8 tensor."""
    image = tf.image.decode_image(image_str_tensor, channels=CHANNELS)
    # Note resize expects a batch_size, but tf_map supresses that index,
    # thus we have to expand then squeeze.  Resize returns float32 in the
    # range [0, uint8_max]
    image = tf.expand_dims(image, 0)
    image = tf.image.resize_bilinear(
        image, [height, width], align_corners=False)
    image = tf.squeeze(image, squeeze_dims=[0])
    image = tf.cast(image, dtype=tf.uint8)
    return image

  def to_img_feature(images):
    """Extract the feature of image vectors"""
    outputs = module(dict(images=images), signature="image_feature_vector", as_dict=True)
    return outputs['default']

  # Step 2) Extract image features of the target image.
  target_image_bytes = tf.gfile.GFile(target_image_path, 'rb').read()
  target_image = tf.constant(target_image_bytes, dtype=tf.string)
  target_image = decode_and_resize(target_image)
  target_image = tf.image.convert_image_dtype(target_image, dtype=tf.float32)
  target_image = tf.expand_dims(target_image, 0)
  target_image = to_img_feature(target_image)

  # Step 3) Extract image features of input images.
  input_byte = tf.placeholder(tf.string, shape=[None])
  input_image = tf.map_fn(decode_and_resize, input_byte, back_prop=False, dtype=tf.uint8)
  input_image = tf.image.convert_image_dtype(input_image, dtype=tf.float32)
  input_image = to_img_feature(input_image)

  # Step 4) Compare cosine_similarities of the target image and the input images.
  dot = tf.tensordot(target_image, tf.transpose(input_image), 1)
  similarity = dot / (tf.norm(target_image, axis=1) * tf.norm(input_image, axis=1))
  similarity = tf.reshape(similarity, [-1])
  
  return input_byte, similarity

In [13]:
import os

path="c:/workspace/project/python-workspace/images"
files = os.listdir(path)
print(files)
paths = [path + "/" + name
         for name in files]
print(paths)

['issue_2023-03-21_102948_edit.png', 'issue_2023-03-21_103145_edit.png', 'issue_2023-03-21_103429.png', 'issue_2023-03-21_103432_edit.png', 'issue_2023-03-21_103627_edit.png', 'Screenshot_20230321_142950_KB.jpg', 'Screenshot_20230321_142959_KB.jpg', 'Screenshot_20230321_143010_KB.jpg', 'Screenshot_20230321_143021_KB.jpg', 'Screenshot_20230321_143035_KB.jpg', 'Screenshot_20230321_143042_KB.jpg']
['c:/workspace/project/python-workspace/images/issue_2023-03-21_102948_edit.png', 'c:/workspace/project/python-workspace/images/issue_2023-03-21_103145_edit.png', 'c:/workspace/project/python-workspace/images/issue_2023-03-21_103429.png', 'c:/workspace/project/python-workspace/images/issue_2023-03-21_103432_edit.png', 'c:/workspace/project/python-workspace/images/issue_2023-03-21_103627_edit.png', 'c:/workspace/project/python-workspace/images/Screenshot_20230321_142950_KB.jpg', 'c:/workspace/project/python-workspace/images/Screenshot_20230321_142959_KB.jpg', 'c:/workspace/project/python-workspac

In [14]:
import time
import tensorflow as tf
from IPython.display import Image, display

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# Load bytes of image files
image_bytes = [tf.io.gfile.GFile(name, 'rb').read()
               for name in paths]

hub_module_url = "https://tfhub.dev/google/imagenet/mobilenet_v2_100_96/feature_vector/1" #@param {type:"string"}

with tf.Graph().as_default():
  input_byte, similarity_op = build_graph(hub_module_url, target_img_path)
  
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    t0 = time.time() # for time check
    
    # Inference similarities
    similarities = sess.run(similarity_op, feed_dict={input_byte: image_bytes})
    
    print("%d images inference time: %.2f s" % (len(similarities), time.time() - t0))

# Display results
print("# Target image")
display(Image(target_img_path))
print("- similarity: %.2f" % similarities[0])

print("# Input images")
for similarity, input_img_path in zip(similarities[1:], input_img_paths):
  display(Image(input_img_path))
  print("- similarity: %.2f" % similarity)

NameError: name 'build_graph' is not defined